## 环境要求

- TensorFlow的版本：2.0 + 
- keras
- sklearn
- librosa

In [1]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载深度学习框架

In [2]:
# 搭建分类模型所需要的库

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## 加载音频处理库

In [1]:
!pip install librosa --user

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 183 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 613 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 1.8 MB/s  eta 0:00:01
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201374 sha256=f57d570bf49fa0e2e2c23bc32c6aa915e7935f2ef6c0b0ed1fb643d75c2eaf9a
  Stored in directory: /Users/sampras/Library/Caches/pip/wheels/c6/08/26/9d7dec1c2dc863d8f1fdc1ac39883e2601dc6fa63fcd68183f
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23142 sha256=1cc9ce52b5227f39fda38261e64321847cf1cbd17bb713b6ce633ae151e7005c
  Stored in directory: /Users/sampras/Library/Caches/pip/wheels/71/76/34/82f346dbd557abb79d415dd4c55281db62cca581ceb1099e0f
  Created wheel for resampy: filename=re

In [3]:
# 其他库

import os
import librosa
import librosa.display
import glob

## 特征提取以及数据集的建立

In [5]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {
    'aloe': 0,
    'burger': 1,
    'cabbage': 2,
    'candied_fruits': 3,
    'carrots': 4,
    'chips': 5,
    'chocolate': 6,
    'drinks': 7,
    'fries': 8,
    'grapes': 9,
    'gummies': 10,
    'ice-cream': 11,
    'jelly': 12,
    'noodles': 13,
    'pickles': 14,
    'pizza': 15,
    'ribs': 16,
    'salmon': 17,
    'soup': 18,
    'wings': 19
}
label_dict_inv = {v: k for k, v in label_dict.items()}

In [21]:
from tqdm import tqdm


def extract_features(parent_dir, sub_dirs, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(
                glob.glob(os.path.join(parent_dir, sub_dir,
                                       file_ext))):  # 遍历数据集的所有文件

            # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,
                                                          sr=sample_rate).T,
                           axis=0)  # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])

    return [feature, label]

In [22]:
# 全部数据参与训练
parent_dir = '/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/train'
save_dir = "/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/"
folds = sub_dirs = np.array([
    'aloe', 'burger', 'cabbage', 'candied_fruits', 'carrots', 'chips',
    'chocolate', 'drinks', 'fries', 'grapes', 'gummies', 'ice-cream', 'jelly',
    'noodles', 'pickles', 'pizza', 'ribs', 'salmon', 'soup', 'wings'
])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir, sub_dirs)

100%|██████████| 298/298 [00:13<00:00, 21.50it/s]


In [23]:
temp = np.array(temp)
data = temp.transpose()

In [24]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：', X.shape)
print('Y的特征尺寸是：', Y.shape)

X的特征尺寸是： (7000, 128)
Y的特征尺寸是： (7000,)


In [26]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [27]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(7000, 128)
(7000, 20)


In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    random_state=1,
                                                    stratify=Y)
print('训练集的大小', len(X_train))
print('测试集的大小', len(X_test))

训练集的大小 5250
测试集的大小 1750


In [29]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

## 建立模型

### 搭建CNN网络

参考：https://www.kaggle.com/leonadoice/task4
> 推荐的资料中，我们推荐大家去看看李宏毅老师的讲的[CNN网络](https://www.youtube.com/watch?v=FrKWiRv254g&feature=youtu.be)这里也附上老师的[PPT](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2017/Lecture/CNN.pdf)。


卷积神经网络CNN的结构一般包含这几个层：

1)输入层：用于数据的输入

2)卷积层：使用卷积核进行特征提取和特征映射------>可以多次重复使用

3)激励层：由于卷积也是一种线性运算，因此需要增加非线性映射(也就是激活函数)

4)池化层：进行下采样，对特征图稀疏处理，减少数据运算量----->可以多次重复使用

5）Flatten操作：将二维的向量，拉直为一维的向量，从而可以放入下一层的神经网络中

6)全连接层：通常在CNN的尾部进行重新拟合，减少特征信息的损失----->DNN网络

In [30]:
model = Sequential()

# 输入的大小
input_dim = (16, 8, 1)

model.add(
    Conv2D(64, (3, 3),
           padding="same",
           activation="tanh",
           input_shape=input_dim))  # 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))  # 最大池化
model.add(Conv2D(128, (3, 3), padding="same", activation="tanh"))  #卷积层
model.add(MaxPool2D(pool_size=(2, 2)))  # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten())  # 展开
model.add(Dense(1024, activation="tanh"))
model.add(Dense(20, activation="softmax"))  # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 4, 2, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             

In [32]:
# 训练模型
model.fit(X_train,
          Y_train,
          epochs=50,
          batch_size=15,
          validation_data=(X_test, Y_test))

Epoch 1/50
350/350 [==============================] - 4s 11ms/step - loss: 2.4940 - accuracy: 0.2547 - val_loss: 2.2654 - val_accuracy: 0.3274
Epoch 2/50
350/350 [==============================] - 4s 11ms/step - loss: 2.0781 - accuracy: 0.3730 - val_loss: 2.0541 - val_accuracy: 0.3943
Epoch 3/50
350/350 [==============================] - 4s 11ms/step - loss: 1.8269 - accuracy: 0.4507 - val_loss: 1.8357 - val_accuracy: 0.4651
Epoch 4/50
350/350 [==============================] - 4s 11ms/step - loss: 1.6086 - accuracy: 0.5173 - val_loss: 1.7449 - val_accuracy: 0.4834
Epoch 5/50
350/350 [==============================] - 4s 11ms/step - loss: 1.4445 - accuracy: 0.5741 - val_loss: 1.7519 - val_accuracy: 0.5034
Epoch 6/50
350/350 [==============================] - 4s 10ms/step - loss: 1.2518 - accuracy: 0.6217 - val_loss: 1.7197 - val_accuracy: 0.5309
Epoch 7/50
350/350 [==============================] - 4s 11ms/step - loss: 1.1179 - accuracy: 0.6659 - val_loss: 1.6393 - val_accuracy: 0.5503

### 预测测试集

In [33]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir,
                                          file_ext))[:]):  # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn, res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,
                       axis=0)  # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [34]:
X_test = extract_features('/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/test/')

100%|██████████| 2000/2000 [01:31<00:00, 21.95it/s]


In [22]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))

In [24]:
preds = np.argmax(predictions, axis=1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('/Users/sampras/Desktop/待学习/NLP/语音识别/数据集/test/*.wav')
result = pd.DataFrame({'name': path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit.csv', index=None)

In [25]:
!ls /Users/sampras/Desktop/待学习/NLP/语音识别/数据集/test/*.wav | wc -l

    2000


In [26]:
!wc -l submit.csv

    2001 submit.csv


In [25]:
result.head() 

NameError: name 'result' is not defined